In [ ]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from PIL import Image, ImageTk
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.models import load_model

# Загрузка модели
#путь к модели
way_to_model = r"D:\WEIGHTS\HAKATON-90"
Model = load_model(way_to_model)

# Размеры изображения, используемые моделью
target_shape = (600, 600)

# Глобальные переменные для изображений
loaded_image_label = None
loaded_image_path = None
image_labels = []

# Функция для классификации изображения и вывода результатов
def classify_image(image_path):
    try:
        # Загрузка и предобработка изображения
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)*255
        image = tf.image.resize(image, target_shape).numpy()
        image = np.resize(image, [1, *target_shape, 3])

        # Классификация изображения
        predictions = Model.predict(image)

        # Отображение результатов
        class_labels = ['Class 1', 'Class 2', 'Class 3']  # Замените на соответствующие метки классов
        result_text.set("Результаты классификации:")
        for i in range(len(class_labels)):
            probability = predictions[0][i]
            label = f"{class_labels[i]}: {probability:.2f}"
            label_text[i].set(label)
    except Exception as e:
        messagebox.showerror("Ошибка", f"Ошибка при обработке изображения: {e}")

# Функция для загрузки изображения
def load_image():
    global loaded_image_label, loaded_image_path

    # Запрос пути к изображению
    image_path = filedialog.askopenfilename(title="Выберите изображение", filetypes=(("JPEG Files", "*.jpg"), ("All Files", "*.*")))
    if image_path:
        # Удаление предыдущего изображения, если оно есть
        if loaded_image_label is not None:
            loaded_image_label.pack_forget()

        # Загрузка нового изображения
        loaded_image_path = image_path
        loaded_image_label = tk.Label(root)
        loaded_image_label.pack()

        # Отображение изображения
        image = Image.open(image_path)
        image = image.resize((200, 200))
        photo = ImageTk.PhotoImage(image)
        loaded_image_label.configure(image=photo)
        loaded_image_label.image = photo

        classify_image(image_path)

# Функция для удаления изображения
def remove_image():
    global loaded_image_label, loaded_image_path

    # Удаление изображения и очистка переменных
    if loaded_image_label is not None:
        loaded_image_label.pack_forget()
        loaded_image_label = None
        loaded_image_path = None

        # Очистка результатов классификации
        result_text.set("Результаты классификации:")
        for i in range(len(class_labels)):
            label_text[i].set("")

# Функция для экспорта результатов в Excel
def export_to_excel():
    # Создание DataFrame с результатами
    data = {'Класс': class_labels, 'Вероятность': [label_text[i].get().split(': ')[1] for i in range(len(class_labels))]}
    df = pd.DataFrame(data)

    # Сохранение DataFrame в Excel
    save_path = filedialog.asksaveasfilename(title="Сохранить результаты", defaultextension=".xlsx", filetypes=(("Excel Files", "*.xlsx"), ("All Files", "*.*")))
    if save_path:
        try:
            df.to_excel(save_path, index=False)
            messagebox.showinfo("Экспорт завершен", "Результаты успешно экспортированы в Excel.")
        except Exception as e:
            messagebox.showerror("Ошибка", f"Ошибка при экспорте в Excel: {e}")

# Создание графического интерфейса
root = tk.Tk()
root.title("Классификатор изображений")

# Кнопка для загрузки изображения
load_button = tk.Button(root, text="Загрузить изображение", command=load_image)
load_button.pack(pady=10)

# Кнопка для удаления изображения
remove_button = tk.Button(root, text="Удалить изображение", command=remove_image)
remove_button.pack(pady=10)

# Отображение результата классификации
result_text = tk.StringVar()
result_label = tk.Label(root, textvariable=result_text)
result_label.pack()

class_labels = ['Кликун', 'Малый', 'Шипун']  # Замените на соответствующие метки классов
label_text = []
for i in range(len(class_labels)):
    label_text.append(tk.StringVar())
    label = tk.Label(root, textvariable=label_text[i])
    label.pack()

# Кнопка для экспорта в Excel
export_button = tk.Button(root, text="Экспорт в Excel", command=export_to_excel)
export_button.pack(pady=10)

root.mainloop()
